# Inference and Evaluation of Fine-tuned CodeT5 for Java Code Summarization

**INSTALL LIBRARIES**
----------------------
----------------------
----------------------
----------------------

In [11]:

!pip install transformers datasets evaluate rouge_score bert_score --quiet


**LOAD HF DATASET**
----------------------
----------------------
----------------------
----------------------

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

dataset = load_dataset("code_x_glue_ct_code_to_text", "java")
# model = GPT2LMHeadModel.from_pretrained("./gpt2-docstring-model").to("cuda" if torch.cuda.is_available() else "cpu")
# tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-docstring-model")


c:\Users\Tanvi\miniconda3\envs\nlpProject\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**INFERENCE**
----------------------
----------------------
----------------------
----------------------

In [6]:
# CodeSummaryGenerator
# This class handles preprocessing, batch inference, and single-example generation for code summarization
# using a fine-tuned CodeT5 model from Hugging Face.

import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
from torch.utils.data import DataLoader
from typing import List, Dict, Optional
from tqdm import tqdm
import os


class CodeSummaryGenerator:
    def __init__(self, model_path: str, decoding_config: Dict, device: Optional[str] = None):
        # Load tokenizer and model from Hugging Face
        self.model_path = model_path
        self.decoding_config = decoding_config
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        if model_path.startswith("pritammane105/GPT2-Code-Summarisation"):
            self.tokenizer = GPT2Tokenizer.from_pretrained(model_path)
            self.model = GPT2LMHeadModel.from_pretrained(model_path).to(self.device)
        else:
            self.tokenizer = AutoTokenizer.from_pretrained(model_path)
            self.model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(self.device)
        # self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        # self.model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(self.device)
        # self.tokenizer = GPT2Tokenizer.from_pretrained(model_path)
        # self.model =GPT2LMHeadModel.from_pretrained(model_path).to("cuda" if torch.cuda.is_available() else "cpu")
        self.model.eval()

    def preprocess_dataset(self, split: str = "validation", max_input_length: int = 512):
        # Load and tokenize dataset split (validation or test)
        dataset = load_dataset("code_x_glue_ct_code_to_text", "java")[split]

        def tokenize_fn(example):
            return self.tokenizer(
                example["code"],
                truncation=True,
                padding="max_length",
                max_length=max_input_length,
            )

        tokenized = dataset.map(tokenize_fn, batched=True)
        tokenized.set_format(type="torch", columns=["input_ids", "attention_mask"])
        return tokenized, dataset["docstring"]

    def generate_summaries(self,
                            tokenized_data,
                            references: List[str],
                            save_path: str,
                            batch_size: int = 16,
                            save_every: int = 50):
        # Perform batch inference on the dataset and periodically save output CSV

        val_loader = DataLoader(tokenized_data, batch_size=batch_size)
        generated_summaries = []
        start_batch = 0

        # Resume if already saved the file with partial results
        if os.path.exists(save_path):
            df_existing = pd.read_csv(save_path)
            generated_summaries = df_existing["predicted_summary"].astype(str).tolist()
            start_batch = len(generated_summaries) // batch_size
            print(f"⏩ Resuming from batch {start_batch} (already {len(generated_summaries)} predictions)")

        for i, batch in enumerate(tqdm(val_loader, desc="Generating Summaries")):
            if i < start_batch:
                continue

            input_ids = batch["input_ids"].to(self.device)
            attention_mask = batch["attention_mask"].to(self.device)

            with torch.no_grad():
                outputs = self.model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    **self.decoding_config
                )

            decoded = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)
            generated_summaries.extend(decoded)

            # Periodically save results to disk
            if (i + 1) % save_every == 0 or (i + 1) == len(val_loader):
                print(f"💾 Saving at batch {i + 1}")
                df = pd.DataFrame({
                    "gold_summary": references[:len(generated_summaries)],
                    "predicted_summary": generated_summaries
                })
                df.to_csv(save_path, index=False)

    def generate_single(self, code_snippet: str):
        # Generate a summary for a single code snippet
        inputs = self.tokenizer(
            code_snippet,
            return_tensors="pt",
            truncation=True,
            max_length=512
        ).to(self.device)

        with torch.no_grad():
            output = self.model.generate(**inputs, **self.decoding_config)

        return self.tokenizer.decode(output[0], skip_special_tokens=True)

**EVALUATION**
----------------------
----------------------
----------------------
----------------------

In [3]:
import evaluate
import numpy as np
from collections import Counter

class SummaryEvaluator:
    def __init__(self):
        self.rouge = evaluate.load("rouge")
        self.bleu = evaluate.load("bleu")
        self.bertscore = evaluate.load("bertscore")

    def avg_token_repetition(self, predictions):
        """
        Computes the average number of repeated tokens per prediction.
        A high repetition score indicates redundancy in the generated text.
        """
        rep_counts = []
        for text in predictions:
            tokens = text.strip().split()
            counts = Counter(tokens)
            repeated_tokens = sum(v for v in counts.values() if v > 1)
            rep_counts.append(repeated_tokens / max(1, len(tokens)))
        return np.mean(rep_counts)

    def evaluate_csvs(self, files: Dict[str, str]):
        """
        Evaluates multiple prediction files and returns a DataFrame of metrics.
        Each file must be a CSV with columns: 'predicted_summary' and 'gold_summary'.
        """
        all_results = []
        for name, path in files.items():
            if not os.path.exists(path):
                print(f"File not found: {path}")
                continue

            df = pd.read_csv(path)
            predictions = df["predicted_summary"].astype(str).tolist()
            references = df["gold_summary"].astype(str).tolist()

            rouge_scores = self.rouge.compute(predictions=predictions, references=references, use_stemmer=True)
            bleu_score = self.bleu.compute(predictions=predictions, references=references)
            bert_score = self.bertscore.compute(predictions=predictions, references=references, lang="en", device="cuda" if torch.cuda.is_available() else "cpu")
            repetition = self.avg_token_repetition(predictions)

            all_results.append({
                "Version": name,
                "ROUGE-1": round(rouge_scores["rouge1"], 4),
                "ROUGE-2": round(rouge_scores["rouge2"], 4),
                "ROUGE-L": round(rouge_scores["rougeL"], 4),
                "BLEU": round(bleu_score["bleu"], 4),
                "BERTScore": round(np.mean(bert_score["f1"]), 4),
                "Avg Token Repetition": round(repetition, 4)
            })

        return pd.DataFrame(all_results)


**INFERENCE & EVALUATION**
----------------------
----------------------
----------------------
----------------------

In [ ]:
# Inference and Evaluation Pipeline for CodeT5
# This script runs multiple decoding strategies on the validation and test sets
# and generates a metrics comparison table with ROUGE, BLEU, BERTScore, Exact Match, and Repetition analysis.

# from inference_pipeline import CodeSummaryGenerator
# from eval_pipeline import SummaryEvaluator


def runConfigs(model_path):
    # Define decoding strategies
    decoding_configs = {
        "baseline_beam": {
            "max_new_tokens": 64,
            "num_beams": 4,
            "early_stopping": True
        },
        "beam_repetition": {
            "max_new_tokens": 64,
            "num_beams": 4,
            "early_stopping": True,
            "repetition_penalty": 1.3
        },
        "sampling_topk": {
            "max_new_tokens": 64,
            "do_sample": True,
            "top_k": 50,
            "temperature": 0.7,
            "repetition_penalty": 1.2
        }
    }
    # Output files (local/Colab environment)
    if model_path=="pritammane105/GPT2-Code-Summarisation":
        output_files = {
            "baseline_beam": "GPT2_val_predictions.csv",
            "beam_repetition": "GPT2_val_beam_repetition.csv",
            "sampling_topk": "GPT2_val_sampling_topk.csv",
            "test_sampling_final": "GPT2_test_sampling_output.csv"
        }
    else:
        output_files = {
            "baseline_beam": "CodeT5_val_predictions.csv",
            "beam_repetition": "CodeT5_val_beam_repetition.csv",
            "sampling_topk": "CodeT5_val_sampling_topk.csv",
            "test_sampling_final": "CodeT5_test_sampling_output.csv"
        }

    # Run decoding strategies for validation set
    for name, config in decoding_configs.items():
        print(f"\n Running config: {name}")
        generator = CodeSummaryGenerator(model_path, config)
        val_tokenized, val_refs = generator.preprocess_dataset("validation")
        generator.generate_summaries(val_tokenized, val_refs, output_files[name])

    print("\n Evaluating generated summaries...")
    evaluator = SummaryEvaluator()
    results_df = evaluator.evaluate_csvs(output_files)
    display(results_df)


runConfigs("pritammane105/GPT2-Code-Summarisation")
runConfigs("pritammane105/CodeT5-Java-Summarisation")




 Running config: baseline_beam


Generating Summaries:   0%|          | 0/324 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


**Miscellaneous Test Samples**
----------------------
----------------------
----------------------
----------------------

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# # Load fine-tuned model from Google Drive
# model_dir = "/content/drive/MyDrive/codet5_checkpoints/checkpoint-46386"

# tokenizer = AutoTokenizer.from_pretrained(model_dir)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

# def generate_summary(code_snippet):
#     inputs = tokenizer(code_snippet, return_tensors="pt", truncation=True, max_length=512).to(model.device)
#     outputs = model.generate(**inputs, max_length=128, num_beams=5, early_stopping=True)
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

# java_code = """
# public int add(int a, int b) {
#     return a + b;
# }
# """
# print(generate_summary(java_code))


Add two integers.

@param a first integer
@param b second integer
@return the sum of the two integers


In [ ]:
java_code = """
public int perform(int a, int b) {
    return a - b;
}
"""
print(generate_summary(java_code))

Computes the difference between two integers.

@param a first integer
@param b second integer
@return the difference


In [ ]:
java_code = """
public int add(int a, int b) {
    return a - b;
}
"""
print(generate_summary(java_code))

Add two integers.

@param a the first integer
@param b the second integer
@return the result


In [ ]:
test_example = dataset["train"][0]
test_code = test_example["code"]
print("ORIGINAL DOCSTRING")
print(test_example["docstring"])
print("\nTESTING CODE:")
print(test_code)

generated_summary = generate_summary(test_code)
print("\nGENERATED SUMMARY:")
print(generated_summary)

ORIGINAL DOCSTRING
Compare the supplied plaintext password to a hashed password.

@param   passwd  Plaintext password.
@param   hashed  scrypt hashed password.

@return true if passwd matches hashed value.

TESTING CODE:
public static boolean check(String passwd, String hashed) {
        try {
            String[] parts = hashed.split("\\$");

            if (parts.length != 5 || !parts[1].equals("s0")) {
                throw new IllegalArgumentException("Invalid hashed value");
            }

            long params = Long.parseLong(parts[2], 16);
            byte[] salt = decode(parts[3].toCharArray());
            byte[] derived0 = decode(parts[4].toCharArray());

            int N = (int) Math.pow(2, params >> 16 & 0xffff);
            int r = (int) params >> 8 & 0xff;
            int p = (int) params      & 0xff;

            byte[] derived1 = SCrypt.scrypt(passwd.getBytes("UTF-8"), salt, N, r, p, 32);

            if (derived0.length != derived1.length) return false;

          

In [ ]:
test_example = dataset["test"][0]
test_code = test_example["code"]
print("ORIGINAL DOCSTRING")
print(test_example["docstring"])
print("\nTESTING CODE:")
print(test_code)

generated_summary = generate_summary(test_code)
print("\nGENERATED SUMMARY:")
print(generated_summary)

ORIGINAL DOCSTRING
Makes sure the fast-path emits in order.
@param value the value to emit or queue up
@param delayError if true, errors are delayed until the source has terminated
@param disposable the resource to dispose if the drain terminates

TESTING CODE:
protected final void fastPathOrderedEmit(U value, boolean delayError, Disposable disposable) {
        final Observer<? super V> observer = downstream;
        final SimplePlainQueue<U> q = queue;

        if (wip.get() == 0 && wip.compareAndSet(0, 1)) {
            if (q.isEmpty()) {
                accept(observer, value);
                if (leave(-1) == 0) {
                    return;
                }
            } else {
                q.offer(value);
            }
        } else {
            q.offer(value);
            if (!enter()) {
                return;
            }
        }
        QueueDrainHelper.drainLoop(q, observer, delayError, disposable, this);
    }

GENERATED SUMMARY:
Fast path emit.

@param value the 

In [ ]:
test_example = dataset["test"][10]
test_code = test_example["code"]
print("ORIGINAL DOCSTRING")
print(test_example["docstring"])
print("\nTESTING CODE:")
print(test_code)

generated_summary = generate_summary(test_code)
print("\nGENERATED SUMMARY:")
print(generated_summary)

ORIGINAL DOCSTRING
Wraps a SingleSource into a Maybe.

<dl>
<dt><b>Scheduler:</b></dt>
<dd>{@code fromSingle} does not operate by default on a particular {@link Scheduler}.</dd>
</dl>
@param <T> the target type
@param singleSource the SingleSource to convert from
@return the new Maybe instance
@throws NullPointerException if single is null

TESTING CODE:
@CheckReturnValue
    @NonNull
    @SchedulerSupport(SchedulerSupport.NONE)
    public static <T> Maybe<T> fromSingle(SingleSource<T> singleSource) {
        ObjectHelper.requireNonNull(singleSource, "singleSource is null");
        return RxJavaPlugins.onAssembly(new MaybeFromSingle<T>(singleSource));
    }

GENERATED SUMMARY:
Construct a Maybe from a SingleSource

<pre>
{@code

import static com.oath.cyclops.reactor.RxJavaPlugins.fromSingle(new SingleSource() {
public void run(Object a) {
if (a != null) {
System.out.println(a);
} else {
System.out.println(a);
}
}
}
}
</pre>


@param singleSource a SingleSource
@param <T> the type of 

In [ ]:
# 🧪 Analysis Script
import pandas as pd
import evaluate
import numpy as np
from datasets import Dataset
from collections import Counter
import os
import torch

# File paths for each decoding strategy
files = {
    "baseline_beam": "/content/drive/MyDrive/codet5_val_predictions.csv",
    "beam_repetition": "/content/drive/MyDrive/codet5_val_beam_repetition.csv",
    "sampling_temp": "/content/drive/MyDrive/codet5_val_sampling_temp.csv"
}

# Load evaluation metrics
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")
bertscore = evaluate.load("bertscore")

# Function to compute average token repetition per summary
def avg_token_repetition(predictions):
    rep_counts = []
    for text in predictions:
        tokens = text.strip().split()
        counts = Counter(tokens)
        repeated_tokens = sum(v for v in counts.values() if v > 1)
        rep_counts.append(repeated_tokens / max(1, len(tokens)))
    return np.mean(rep_counts)

# Run metrics for each version
all_results = []

for name, path in files.items():
    if not os.path.exists(path):
        print(f" File not found: {path}")
        continue

    df = pd.read_csv(path)
    predictions = df["predicted_summary"].astype(str).tolist()
    references = df["gold_summary"].astype(str).tolist()

    # Compute metrics
    rouge_scores = rouge.compute(predictions=predictions, references=references, use_stemmer=True)
    bleu_score = bleu.compute(
        predictions=predictions,
        references=references
    )
    bert_score = bertscore.compute(predictions=predictions, references=references, lang="en", device="cuda")
    exact_match = sum(p.strip() == r.strip() for p, r in zip(predictions, references)) / len(references)
    repetition = avg_token_repetition(predictions)

    all_results.append({
        "Version": name,
        "ROUGE-1": round(rouge_scores["rouge1"], 4),
        "ROUGE-2": round(rouge_scores["rouge2"], 4),
        "ROUGE-L": round(rouge_scores["rougeL"], 4),
        "BLEU": round(bleu_score["bleu"], 4),
        "BERTScore": round(np.mean(bert_score["f1"]), 4),
        "Exact Match": round(exact_match, 4),
        "Avg Token Repetition": round(repetition, 4)
    })

# Display comparison table
results_df = pd.DataFrame(all_results)
results_df = results_df.sort_values("ROUGE-L", ascending=False)
display(results_df)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


,Version,ROUGE-1,ROUGE-2,ROUGE-L,BLEU,BERTScore,Exact Match,Avg Token Repetition
1,beam_repetition,0.4677,0.2558,0.4205,0.1643,0.8840,0.0068,0.4734
0,baseline_beam,0.4094,0.2245,0.3648,0.1415,0.8667,0.0058,0.5815
2,sampling_temp,0.4059,0.1746,0.3348,0.1224,0.8723,0.0014,0.2246


In [ ]:
# SCRIPT 2: Sampling with Temperature + Top-k
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import pandas as pd
import os

# Configs
model_dir = "/content/drive/MyDrive/codet5_checkpoints/checkpoint-46386"
save_path = "/content/drive/MyDrive/codet5_val_sampling_temp.csv"
save_every = 50

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir).to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

dataset = load_dataset("code_x_glue_ct_code_to_text", "java")
val_data = dataset["validation"]
gold_summaries = val_data["docstring"]

def tokenize_fn(example):
    return tokenizer(example["code"], truncation=True, padding="max_length", max_length=512)

val_tokenized = val_data.map(tokenize_fn, batched=True)
val_tokenized.set_format(type="torch", columns=["input_ids", "attention_mask"])
val_loader = DataLoader(val_tokenized, batch_size=16)

generated_summaries = []
start_batch = 0

if os.path.exists(save_path):
    df_existing = pd.read_csv(save_path)
    generated_summaries = df_existing["predicted_summary"].tolist()
    start_batch = len(generated_summaries) // 16
    print(f"Resuming from batch {start_batch} (already {len(generated_summaries)} predictions)")

for i, batch in enumerate(tqdm(val_loader, desc="Generating (Sampling + Temp)")):
    if i < start_batch:
        continue

    input_ids = batch["input_ids"].to(model.device)
    attention_mask = batch["attention_mask"].to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            do_sample=True,
            top_k=50,
            temperature=0.7,
            max_new_tokens=64,
            repetition_penalty=1.2,
            early_stopping=True
        )

    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    generated_summaries.extend(decoded)

    if (i + 1) % save_every == 0 or (i + 1) == len(val_loader):
        print(f"Saving at batch {i + 1}")
        df = pd.DataFrame({
            "gold_summary": gold_summaries[:len(generated_summaries)],
            "predicted_summary": generated_summaries
        })
        df.to_csv(save_path, index=False)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/26.7k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/141M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/4.25M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/9.38M [00:00<?, ?B/s]

...

Saving at batch 250


Generating (Sampling + Temp):  93%|█████████▎| 300/324 [09:19<00:48,  2.01s/it]

Saving at batch 300


Generating (Sampling + Temp): 100%|██████████| 324/324 [10:03<00:00,  1.86s/it]

Saving at batch 324
